<a href="https://colab.research.google.com/github/D1emonk/My-repo/blob/dev/Lesson_22_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neptune

In [ ]:
!pip install optuna

In [ ]:
!pip install neptune_tensorflow_keras

In [ ]:
!pip install -U neptune neptune-tensorflow-keras

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback
import shap
import matplotlib.pyplot as plt

In [6]:
# Инициализация Neptune
run = neptune.init_run(
    project="d1emonk/tutorial",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJjYjQ2YTk1Yy00OTllLTQ2ZTctOTAyZi02OGU4NmVhM2Q4NzkifQ==",
)

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/d1emonk/tutorial/e/TUT-5


In [7]:
# Загрузка данных
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/loan_data.csv')

Mounted at /content/drive


In [8]:
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [10]:
df_factorized = pd.DataFrame(df)
for column in df_factorized.select_dtypes(include=['object']).columns:
    df_factorized[column], unique = pd.factorize(df_factorized[column])

In [11]:
# Разделение данных
X = df_factorized.drop('loan_status', axis=1).values
y = df_factorized['loan_status'].values

In [12]:
# Стратифицированное разделение
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, stratify=y_train, random_state=42
)

In [13]:
# Нормализация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [14]:
# Расчет весов классов
class_counts = np.bincount(y_train)
total = len(y_train)
class_weights = {0: total/(2*class_counts[0]), 1: total/(2*class_counts[1])}

In [15]:
# Метрика F1
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + 1e-6))

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()

In [16]:
# Функция создания модели
def create_model(trial):
    params = {
        'num_layers': trial.suggest_int('num_layers', 1, 3),
        'units': trial.suggest_categorical('units', [64, 128, 256]),
        'dropout': trial.suggest_float('dropout', 0.2, 0.5),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-2),
        'activation': trial.suggest_categorical('activation', ['relu', 'leaky_relu']),
        'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128])
    }

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X_train.shape[1],)))

    for _ in range(params['num_layers']):
        model.add(tf.keras.layers.Dense(params['units'], activation=params['activation']))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(params['dropout']))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy', F1Score(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model, params

In [17]:
# Функция для Optuna
def objective(trial):
    model, params = create_model(trial)

    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='val_f1_score',
        patience=5,
        mode='max',
        restore_best_weights=True
    )

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=params['batch_size'],
        class_weight=class_weights,
        callbacks=[early_stop, NeptuneCallback(run)],
        verbose=0
    )

    # Логирование параметров
    run["parameters"] = params
    run["best_val_f1"] = max(history.history['val_f1_score'])

    return max(history.history['val_f1_score'])

In [18]:
# Оптимизация гиперпараметров
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, timeout=3600)

[I 2025-03-24 07:24:19,730] A new study created in memory with name: no-name-9c114678-9c57-4100-aa0e-e704d2154fd2
<ipython-input-16-d3f3785d4e9c>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-2),


[I 2025-03-24 07:25:08,064] Trial 0 finished with value: 0.7560763359069824 and parameters: {'num_layers': 1, 'units': 256, 'dropout': 0.22746301520126136, 'learning_rate': 0.00021015793097162297, 'activation': 'relu', 'batch_size': 32}. Best is trial 0 with value: 0.7560763359069824.


[I 2025-03-24 07:25:43,738] Trial 1 finished with value: 0.7463808655738831 and parameters: {'num_layers': 1, 'units': 128, 'dropout': 0.47458860450771223, 'learning_rate': 0.00033482804680194216, 'activation': 'leaky_relu', 'batch_size': 128}. Best is trial 0 with value: 0.7560763359069824.


[I 2025-03-24 07:26:33,506] Trial 2 finished with value: 0.7686495780944824 and parameters: {'num_layers': 2, 'units': 64, 'dropout': 0.2407061111731763, 'learning_rate': 0.0011215707574165862, 'activation': 'relu', 'batch_size': 32}. Best is trial 2 with value: 0.7686495780944824.


[I 2025-03-24 07:27:07,095] Trial 3 finished with value: 0.7546820640563965 and parameters: {'num_layers': 2, 'units': 64, 'dropout': 0.423502947686036, 'learning_rate': 0.00660948618941375, 'activation': 'leaky_relu', 'batch_size': 32}. Best is trial 2 with value: 0.7686495780944824.


...

[I 2025-03-24 07:47:47,362] Trial 25 finished with value: 0.7598001956939697 and parameters: {'num_layers': 2, 'units': 64, 'dropout': 0.38225015087118674, 'learning_rate': 0.0037956525359255145, 'activation': 'relu', 'batch_size': 32}. Best is trial 24 with value: 0.7816569209098816.


[I 2025-03-24 07:48:36,766] Trial 26 finished with value: 0.7569226026535034 and parameters: {'num_layers': 3, 'units': 64, 'dropout': 0.39642540103757185, 'learning_rate': 0.006644629521412986, 'activation': 'leaky_relu', 'batch_size': 32}. Best is trial 24 with value: 0.7816569209098816.


[I 2025-03-24 07:50:01,358] Trial 27 finished with value: 0.7629978060722351 and parameters: {'num_layers': 3, 'units': 64, 'dropout': 0.4849530455359349, 'learning_rate': 0.0014106016093094445, 'activation': 'relu', 'batch_size': 32}. Best is trial 24 with value: 0.7816569209098816.


[I 2025-03-24 07:50:33,127] Trial 28 finished with value: 0.7654261589050293 and parameters: {'num_layers': 2, 'units': 64, 'dropout': 0.4529071645591746, 'learning_rate': 0.00924123742377359, 'activation': 'relu', 'batch_size': 32}. Best is trial 24 with value: 0.7816569209098816.


[I 2025-03-24 07:52:29,941] Trial 29 finished with value: 0.7854382991790771 and parameters: {'num_layers': 3, 'units': 256, 'dropout': 0.33751962292886006, 'learning_rate': 0.0032030823775254843, 'activation': 'relu', 'batch_size': 32}. Best is trial 29 with value: 0.7854382991790771.


In [19]:
# Получаем лучшие параметры
best_params = study.best_params

In [20]:
# Проверяем batch_size
if isinstance(best_params['batch_size'], str):
    if best_params['batch_size'] == 'auto':
        batch_size = 32  # Установите значение по умолчанию
    else:
        raise ValueError(f"Неподдерживаемое значение batch_size: {best_params['batch_size']}")
else:
    batch_size = int(best_params['batch_size'])

In [21]:
print("Лучшие параметры:", study.best_params)

Лучшие параметры: {'num_layers': 3, 'units': 256, 'dropout': 0.33751962292886006, 'learning_rate': 0.0032030823775254843, 'activation': 'relu', 'batch_size': 32}


In [22]:
# Обучение модели
best_model, _ = create_model(study.best_trial)
best_model.fit(
    np.concatenate([X_train, X_val]),
    np.concatenate([y_train, y_val]),
    epochs=100,
    batch_size=int(study.best_params['batch_size']),
    class_weight=class_weights,
    callbacks=[NeptuneCallback(run)]
)

<ipython-input-16-d3f3785d4e9c>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-2),


ValueError: invalid literal for int() with base 10: 'auto'

In [ ]:
# Оценка на тестовых данных
y_pred = (best_model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

In [ ]:
run.stop()